# Validação de Boleto ITAÚ

API de RPA para validação do Boleto Automatico, por meio de importação de dados na Planilha "validação.xlsx". 

In [1]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
from smtplib import SMTP
import pandas as pd
import numpy as nd
import html
import PyPDF2 
from PyPDF2 import PdfFileReader, PdfFileMerger, PdfMerger
import re

#Tabelas

tabela = pd.read_excel ("validacao.xlsx") 
validação_df = pd.read_excel("validacao.xlsx")
validação_df2 = validação_df.dropna()
validação_importada = pd.ExcelWriter("validacao_importada.xlsx")
validação_df2.to_excel(validação_importada,'importados', index=False)
validação_importada.save()
validação_limpa = pd.read_excel("validacao_importada.xlsx")
contagem = len(validação_limpa.axes[0])
lin = 0 

###TRATANDO PLANILHAS###
while lin <= contagem:

    BANCO = validação_limpa.loc[lin,'Banco']
    EMPRESA = validação_limpa.loc[lin,'Empresa']
    CNPJ = validação_limpa.loc[lin,'cnpj']
    AG = validação_limpa.loc[lin,'agencia']
    CC = validação_limpa.loc[lin,'conta']
    CON = validação_limpa.loc[lin,'convenio']
    ARQ_PBM = validação_limpa.loc[lin,'arquivo-pbm']
    ARQ_BANCO = validação_limpa.loc[lin,'arquivo-banco']
    EMAIL = validação_limpa.loc[lin,'email']
    CLIENTE = validação_limpa.loc[lin,'Cliente']
    recipients = [EMAIL,CLIENTE]
    assinatura = '<div>Muito obrigado pela atenção, qualquer duvida estou à disposição</div><div>Atenciosamente,</div><div>&nbsp;</div><div> <table border="0" cellspacing="0" cellpadding="0"> <tbody> <tr> <td valign="top" width="100"><img src="https://finnet.com.br/wp-content/uploads/2020/10/cropped-favicon.png" alt="image.png" width="96" height="90"/><span style="color: #888888;"><br/></span></td><td valign="top"> <table border="0" cellspacing="0" cellpadding="0"> <tbody> <tr> <td> <div><strong>Otavio Lopes Feitosa<br/></strong></div></td></tr><tr> <td> <div><span>Analista de implantação/ Onboard</span></div></td></tr><tr> <td>(11) 3882-9200</td></tr><tr> <td><span><a href="http://www.finnet.com.br/" rel="noopener" target="_blank">www.finnet.com.br</a></span></td></tr><tr> <td><a href="mailto:otavio.feitosa@finnet.com.br" target="_blank">otavio.feitosa@finnet.com.br</a></td></tr></tbody> </table> </td><td valign="middle"> <table border="0" cellspacing="0" cellpadding="0"> <tbody> <tr> <td><a href="https://www.linkedin.com/company/finnet-/" target="_blank"><img src="https://ci4.googleusercontent.com/proxy/Q_Mjay34V4NOoLQ4oj0xFeVUc2zKZWqLlP1y12Xcr4Dwyy_kI7C_ngpL8Qd7u5VM9w=s0-d-e1-ft#https://i.ibb.co/H76MTPw/1.jpg" alt="" width="22"/></a></td></tr><tr> <td><a href="https://www.facebook.com/finnetbrasil/" target="_blank"><img src="https://ci6.googleusercontent.com/proxy/wz7ymU3fGS_VhPvAhf2lpujzWWp9pMyy74V7jgqHQ9XgX9s2vyXZkxtNCJAn5ypRzA=s0-d-e1-ft#https://i.ibb.co/7G9PMMk/2.jpg" alt="" width="22" height="22"/></a></td></tr><tr> <td><a href="https://www.instagram.com/Finnetbrasil/" target="_blank"><img src="https://ci3.googleusercontent.com/proxy/MH5PC5K28BonY589PrtqJ9CDQbIoHjUjL-6ToN8zo6s-BVUqZVPraN9LBkFP5JOcHQ=s0-d-e1-ft#https://i.ibb.co/qYFVgPf/3.jpg" alt="" width="22" height="22"/></a></td></tr></tbody> </table> </td></tr></tbody> </table> </div><div>&nbsp;</div><div> <div> <div><strong><em>Finnet: Aliando tecnologia financeira &agrave; gera&ccedil;&atilde;o de cr&eacute;dito, por meio do maior marketplace do Brasil, com acesso &agrave;s maiores institui&ccedil;&otilde;es financeiras.</em></strong></div></div></div>'

        ###BOLETO PBM###
    #Lendo arquivo em forma binaria 
    PBMbi = open(ARQ_PBM,'rb')
    #Read file no arquivo binario
    Boletobi = PyPDF2.PdfFileReader(PBMbi)
    Boletobi = Boletobi.getPage(0)
    BoletoPBM = Boletobi.extractText()
    
    #Buscando validadores variaveis
    busca_linha_PBM = re.findall("\d{5}\.\d{5} \d{5}\.\d{6} \d{5}\.\d{6} \d \d{14}", BoletoPBM)


    ###Validadores###
    linha = busca_linha_PBM[0]
    linha4 = BoletoPBM.split('\n')[4]
    num_doc = linha4.split(' ')[1]
    print(f"Numero de Doucmento: {num_doc}")
    Ag_noss_doc = linha4.split(' ')[5]
    Ag = Ag_noss_doc.split('/')[0]
    print(f"Numero de Agência: {Ag}")
    noss_doc = Ag_noss_doc.split('/')[1]
    print(f"Nosso Numero: {noss_doc}")
    validação = 0
    
    
        #Lendo arquivo do banco binaria 
    O2viabi = open(ARQ_BANCO,'rb')
    #Read file no arquivo binario
    Boletobi = PyPDF2.PdfFileReader(O2viabi)
    Boletobi = Boletobi.getPage(0)
    BoletoBNC = Boletobi.extractText(0)
    
    ##Validando linha nos boletos###
    if linha in BoletoBNC:
        validação += 1
        ###validando numero de documento###
        if num_doc in BoletoBNC :
            validação += 1
            if Ag in BoletoBNC :
                validação += 1
                if noss_doc in BoletoBNC :
                    validação += 1
                    if validação == 4:
                        print("Boleto validados")
                        pdfs = [ARQ_PBM, ARQ_BANCO]
                        merger = PdfMerger()
                        for pdf in pdfs:
                            merger.append(pdf)
                        ###Merger###
                        merger.write("validacao-finnet.pdf")
                        merger.close()
                                                             ###DEFININDO SERVIDOR###
                        rost = "smtp.gmail.com"
                        port = "587"
                        login = "otavio.feitosa@finnet.com.br"
                        senha = "Finnet@2021"
                                                    ###ABRINDO SERIDOR###
                        server: SMTP = smtplib.SMTP(rost, port)
                        smtplib.SMTP_SSL(rost, 465)
                        server.ehlo()
                        server.starttls()
                        server.login(login, senha)
                                    ###DEFININDO CORPO DO E-MAIL
                        corpo = f"""
                        <img src="https://i.imgur.com/qnyk2bT.png" width="700px"/>
                        <p>Prezado cliente,</p> 
                        <p>Envio anexo um boleto do banco {BANCO} gerado pelo Portal de Boletos.</p>
                        <p>Realizamos a comparação do boleto gerado no Portal com o boleto que nos enviaram.</p>
                        <p>📌 Por gentileza, para que possamos realizar a virada desta conta em produção, precisamos da sua validação sobre o boleto do Portal e do seu de acordo formal em resposta a este e-mail.<p>
                        <p>Qualquer dúvida ou divergência identificada, por favor nos comunique imediatamente.<p>
                        <p>Ficamos no aguardo de um breve retorno.<p>
                        <p>Atenciosamente,<p>
                        <div style='background:yellow;width:540px;'>
                        <p>📌Por gentileza após a validação, informar uma previsão para o envio de um arquivo de remessa para o acompanhamento do arquivo em produção.</p>
                        </div>
                        {assinatura}
                        """
                        email_msg = MIMEMultipart()
                                                        ###Definindo destinatarios e configurações###
                        email_msg['From'] = login
                        email_msg['To'] = ", ".join(recipients)
                        email_msg['Subject'] = f"Validação de boleto | {BANCO} | {EMPRESA}"
                        email_msg.attach(MIMEText(corpo,'html'))
                                                        ###abrir arquivos em bi
                        cam = "C:\\Users\\otavio.feitosa\\Desktop\\OTAVIO\\CURSOS\\PYTHON\\INTENSIVÃO\\AULA1\\validacao-finnet.pdf"
                        attchment = open(cam,'rb')
                                                        ###ler arquivo binario e codificado em base 64
                        att = MIMEBase('application', 'octet-stream')
                        att.set_payload(attchment.read())
                        encoders.encode_base64(att) 
                        boletos = "validacao-finnet.pdf"
                        att.add_header('Content-Disposition',f'att; filename= validacao-finnet.pdf')
                        attchment.close()
                        email_msg.attach(att)
                                                        ###Enviar e-mail###
                        server.sendmail(email_msg['From'],email_msg['To'],email_msg.as_string())
                        lin += 1
                        print(f"Validação enviada para {EMPRESA}")
                        
                    else:
                        ("Boletos não são validos")
                        lin += 1
                else:
                    prit("Nosso numero não bate")
                    lin += 1
            else:
                print("Agencia não bate")
                lin += 1
        else:
            print("numero de documento não bate")
            lin += 1
    else:
        print("linha digitavel não bate")
        lin += 1

C:\Users\otavio.feitosa\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Numero de Doucmento: 4630001
Numero de Agência: 109
Nosso Numero: 01367348-8
Boleto validados
Validação enviada para SBB


FileNotFoundError: [Errno 2] No such file or directory: 'boleto-PBM.pdf'